In [3]:
from datasets import load_dataset, load_metric
dataset = load_dataset("swag", "regular")

Reusing dataset swag (C:\Users\Daniel\.cache\huggingface\datasets\swag\regular\0.0.0\f9784740e0964a3c799d68cec0d992cc267d3fe94f3e048175eca69d739b980d)


In [6]:
import utils.visualization as visualization
x= dataset['train']
visualization.show_random_elements(x)

,ending0,ending1,ending2,ending3,fold-ind,gold-source,label,sent1,sent2,startphrase,video-id
0,continues taking off the snow and is pulled off top.,runs into the camera.,turns around and goes back to the other room.,goes off and back.,2272,gold,2,A dog in socks walks slowly out onto the floor as a lady films him.,The dog,A dog in socks walks slowly out onto the floor as a lady films him. The dog,anetv_ogQozSI5V8U
1,puts his feet down on the stage in front of the audience while a band behind him talks to one another.,vigorously about to hit someone with his hands as other dancers pose offscreen for him.,"is now watching them as the girls performance and hold each other after singing, dancing, dancing, and twirling and singing.",remains in the front of the triangle formation and continues to dance while the two behind her mimics her every move.,17606,gold,3,All of the ladies begin to dance in unison moving their hands from left to right and jumping from side to side.,The instructor,All of the ladies begin to dance in unison moving their hands from left to right and jumping from side to side. The instructor,anetv_31TT2oiYRO4
2,finishes her routine while flipping.,stretches her legs slightly.,holds the bar up to her height.,does push ups on the bar.,19781,gen,1,The girl stands to her feet.,the girl,The girl stands to her feet. the girl,anetv_TxYZLJQOHvY
3,swims through this wavy section.,"drains his hair, then climbs over and turns to cuff her.",looks back at the house.,"wraps his arms around someone, who now sits down and gently cups his hands with hers.",344,gen,3,She comes over and takes someone's hand.,Someone,She comes over and takes someone's hand. Someone,lsmdc1044_Pride_And_Prejudice_Disk_Two-89595
4,is not walking toward him.,breaks suddenly and backs up.,lets it into the elevator.,arrives at the motel center entrance and runs the street back holding a upside - upon straw.,17140,gold,1,"The middle - aged man offers his keys nervously as someone drives down the street towards someone, and stops as someone slaps the bonnet off the Spanish police car.",Someone,"The middle - aged man offers his keys nervously as someone drives down the street towards someone, and stops as someone slaps the bonnet off the Spanish police car. Someone",lsmdc1043_Vantage_Point-89397
5,get back into their car and put the snowboard on top of the car.,are racing in trash on the snow in front of them.,are in the snow above snow from the mountain.,are racing down a hill with swirling snow and cleaning them from the water.,16325,gold,0,People are snowboarding down a hill of snow.,People,People are snowboarding down a hill of snow. People,anetv_vu65aIIJHtU
6,"gives a nod, brightens his eyes as someone looks through a tiny window.",lunges out of his way.,helps someone by lifting up the jug from someone's face.,looks up from the poster.,18204,gen,1,Someone nervously faces off with someone.,Someone,Someone nervously faces off with someone. Someone,lsmdc3042_KARATE_KID-20190
7,stands fully packed with boats.,"is clear and bright, the sun shining over the skyscrapers of manhattan.",is a hydra production line as he listens in.,", she now runs through the apartment gates and outside into the evening.",4925,gold,1,"She has the broadest of grins as she stands in the pouring rain, looking up into the sky.",The next day,"She has the broadest of grins as she stands in the pouring rain, looking up into the sky. The next day",lsmdc1007_Spider-Man1-74761
8,drops four more forms.,push the side of the roof and is interviewed on the screen.,goes steadily while rollerblading.,pulls out some keys with a bat.,5413,gold,2,Two people are doing figure eights on a bridge as they roller blade.,One person,Two people are doing figure eights on a bridge as they roller blade. One person,anetv_Ffi7vDa3C2I
9,bouncing in the park in the snow.,standing up from the ladder.,skiing down slope with is knees bend down.,pulled on the kayak beside boat.,8133,gold,2,The man in red jacket is ski

In [7]:
from utils.model_loading import get_model_and_tokenizer_for_classification
import  utils.datasets_loading as datasets_loading

#Load model for classification
model, tokenizer = get_model_and_tokenizer_for_classification()

a = x[:2]
#tokenize the data into vectors
processed = datasets_loading.preprocess_function_swag(a, tokenizer)

print(a)
#print the vectors
print(processed)
#decode back to see we get the original sentences
print([tokenizer.decode(processed['input_ids'][i]) + '\n' for i in range(4)] )

Some weights of the model checkpoint at distilbert-base-uncased-distilled-squad were not used when initializing DistilBertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-distilled-squad and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

{'ending0': ['passes by walking down the street playing their instruments.', 'are playing ping pong and celebrating one left each in quick.'], 'ending1': ['has heard approaching them.', 'wait slowly towards the cadets.'], 'ending2': ["arrives and they're outside dancing and asleep.", 'continues to play as well along the crowd along with the band being interviewed.'], 'ending3': ['turns the lead singer watches the performance.', 'continue to play marching, interspersed.'], 'fold-ind': ['3416', '3417'], 'gold-source': ['gold', 'gen'], 'label': [0, 3], 'sent1': ['Members of the procession walk down the street holding small horn brass instruments.', 'A drum line passes by walking down the street playing their instruments.'], 'sent2': ['A drum line', 'Members of the procession'], 'startphrase': ['Members of the procession walk down the street holding small horn brass instruments. A drum line', 'A drum line passes by walking down the street playing their instruments. Members of the processio

now run training on tiny set

In [ ]:
import utils.datasets_loading as datasets_loading
encoded_dataset = datasets_loading.preprocess_swag(dataset,tokenizer)

from transformers import TrainingArguments, Trainer

metric_name = "accuracy"
batch_size = 12

benchmark_folder_name = "swag-classification"
args = TrainingArguments(
    benchmark_folder_name,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    #todo fix one use accuracy metric(use sklearn)
    # metric_for_best_model=metric_name,
)

#todo fix accuracy
# metric = load_metric(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return predictions == labels


validation_key = "validation"

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"].select(range(20)),
                                #todo fit on real validation data
    eval_dataset=encoded_dataset["train"].select(range(20)),
    # eval_dataset=encoded_dataset[validation_key].select(range(10)),
    tokenizer=tokenizer,
    #todo fix accyracy
    # compute_metrics=compute_metrics
)

trainer.train()




preprocess_swag took  42.20380163192749 seconds


Epoch,Training Loss,Validation Loss
